## make data for history endpoints

In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)

In [3]:
#create dataframe
brew_df = pd.read_sql_query("SELECT * FROM business", con=engine)
brew_df

,index,Company Name,street,City,State,Zip Code,Zip Plus,Contact First Name,Contact Last Name,Contact Title,...,NAICS2,NAICS3,NAICS4,NAICS5,Business Type,County,Employee,Annual Sales,URL,Year Established
0,0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,...,NaN,NaN,NaN,NaN,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016
1,1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,None,None,None,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,None,2018
2,2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,None,None,None,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,None,2017
3,3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,None,None,None,...,NaN,NaN,NaN,NaN,Private,Cherokee,14,17916000.0,None,2018
4,4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,...,NaN,NaN,NaN,NaN,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,Wise Man Brewing,826 Angelos Bros Ave,Winston Salem,NC,27101,1524.0,Michael D,Beverly,Manager,...,NaN,NaN,NaN,NaN,Private,Forsyth,14,5870000.0,WISEMANBREWING.COM,2017
272,272,Wooden Robot Brewery LLC,1440 S Tryon St # 110,Charlotte,NC,28203,4263.0,Joshua,Patton,CEO,...,NaN,NaN,NaN,NaN,Private,Mecklenburg,14,7049000.0,WOODENROBOTBREWERY.COM,2015
273,273,Yesteryears Brewery,300 E Main St # C,Carrboro,NC,27510,2359.0,David,Larsen,Manager,...,NaN,NaN,NaN,NaN,Private,Orange,14,21819000.0,None,2016
274,274,York Chester Brewing Co,1500 River Dr # 250,Belmont,NC,28012,3587.0,None,None,None,...,NaN,NaN,NaN,NaN,Private,Gaston,14,20830000.0,YORKCHESTERBREW.COM,2017


In [4]:
# create database of breweries founded per year
brew_df.loc[brew_df['Year Established'] <= 1990, 'Year Established'] = 1990
yeargrouped = pd.DataFrame(brew_df.groupby('Year Established').count()['Company Name']).reset_index()
yeargrouped = yeargrouped.rename(columns={'Year Established':'yearEstablished', 'Company Name':'yearCount'})
yeargrouped['cumulativeCount'] = yeargrouped['yearCount'].cumsum()
yeargrouped

,yearEstablished,yearCount,cumulativeCount
0,1990,4,4
1,1991,1,5
2,1995,1,6
3,1996,2,8
4,1997,2,10
5,1999,2,12
6,2001,3,15
7,2003,2,17
8,2004,2,19
9,2005,1,20


In [5]:
# add data to sqlite database
yeargrouped.to_sql('history', con=engine, if_exists='replace')